In [ ]:
!pip install torch transformers gradio flask

In [ ]:
import re
import torch
import gradio as gr
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    AutoConfig,
    pipeline,
    logging
)

# Optional: reduce logging verbosity.
logging.set_verbosity_error()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ---------------- Ethical Filtering ----------------
banned_words = ["hate", "kill", "stupid", "idiot", "terrorist", "bomb", "nazi", "racist", "sexist"]

def contains_banned_words(text: str) -> bool:
    text = text or ""
    text_lower = text.lower()
    for word in banned_words:
        if re.search(r"\b" + re.escape(word) + r"\b", text_lower):
            return True
    return False

# ---------------- Sentiment Analysis ----------------
sentiment_pipeline = None  # Load sentiment model


# ---------------- Model Loading ----------------
base_checkpoint = "google/flan-t5-large"

# Load the base model and tokenizer.
tokenizer = None
model = None

# ---------------- Response Function ----------------
def respond(message: str,
            history: list,  # history is managed automatically by Gradio.
            system_message: str,
            max_tokens: int,
            temperature: float,
            top_p: float):
    """
    Respond to the user's message.
    - Applies ethical filtering and sentiment checks.
    - Builds a prompt using the system message and current user message.
    - Generates a response using our LoRA-finetuned FLAN-T5-large model.
    Returns a single string reply.
    """
    if message is None or message.strip() == "":
        return ""

    # Ethical filtering.
    None

    # Sentiment analysis.
    sentiment = sentiment_pipeline(message)[0]
    if sentiment["label"] == "NEGATIVE" and sentiment["score"] > 0.90:
        return "I sense some negativity in your message. Let's try to keep our conversation respectful."

    # Build prompt: use the system message plus the current user message.
    prompt = None # Build Prompt
    inputs = None # Tokenize
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Ensure sampling parameters are in effect.
    outputs = None # Generate Answer
    response = None # Decode
    # Remove the prompt from the output if it's there.
    if response.startswith(prompt):
        response = response[len(prompt):].strip()
    return response

# ---------------- Gradio Chat Interface ----------------
demo = gr.ChatInterface(
    fn=respond,
    type="messages",  # This indicates that the conversation history is maintained by Gradio.
    title="Ethical Chatbot",
    description="Chatbot using a FLAN-T5-large model for responses and DistilBERT for sentiment analysis with ethical filtering.",
    additional_inputs=[
        gr.Textbox(value="You are a friendly Chatbot. Please respond helpfully.", label="System message"),
        gr.Slider(minimum=1, maximum=2048, value=150, step=1, label="Max new tokens"),
        gr.Slider(minimum=0.1, maximum=4.0, value=0.7, step=0.1, label="Temperature"),
        gr.Slider(minimum=0.1, maximum=1.0, value=0.95, step=0.05, label="Top-p (nucleus sampling)")
    ]
)

if __name__ == "__main__":
    demo.launch(debug=True, share=True)
